# Neural Network Word-Level Generator



In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
%cd ..
%cd gdrive/My\ Drive/NLP\ Final\ Project

/
/gdrive/My Drive/NLP Final Project


# Generate the Sequences

In [ ]:
import string
# Generates a file to hold sequences of text
# Don't need to run this again
def sequences(file):
  train = open(file, 'r')
  text = train.read()
  train.close()

  # test = text.replace(' --', '')

  words = text.split()

  # table = str.maketrans('', '', string.punctuation)
  # words = [w.translate(table) for w in words]

  length = 51
  sequences = list()
  for i in range(length, len(words)):
    seq = words[i-length: i]
    line = ' '.join(seq)
    sequences.append(line)
  
  doc = '\n'.join(sequences)
  with open('obama_sequences.txt', 'w') as output:
    output.write(doc)
    print('making doc')

sequences('train.txt')

making doc


# Train the Neural Model

In [3]:
# Open the sequences doc
def open_doc(file):
  doc = open(file, 'r')
  text = doc.read()
  doc.close()
  return text

In [ ]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding


# open the file and split into lines
file = open_doc('obama_sequences.txt')
lines = file.split('\n')

# tokenize and fit the model, then turn the lines into sequences
tokenizer = Tokenizer(filters=' ')
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

# vocab_size is length of the tokens (accessed by tokenizer.word_index) plus one due to weird indexing
vocab_size = len(tokenizer.word_index)+1

# separate the sequences into input an output
sequences = array(sequences)
# X is input sequence (first 50 words), y is output word(51st word)
X, y = sequences[:,:-1], sequences[:,-1]
# one hot encodes the output word (ie 51st word)
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

# define the model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
# print(model.summary())

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=90)

model.save('model-90.h5')
dump(tokenizer, open('tokenizer-90.pkl', 'wb'))

Epoch 1/90
447/447 [==============================] - 94s 210ms/step - loss: 6.4987 - accuracy: 0.0486
Epoch 2/90
447/447 [==============================] - 89s 200ms/step - loss: 6.0571 - accuracy: 0.0690
Epoch 3/90
447/447 [==============================] - 93s 208ms/step - loss: 5.8383 - accuracy: 0.0792
Epoch 4/90
447/447 [==============================] - 94s 210ms/step - loss: 5.6613 - accuracy: 0.0872
Epoch 5/90
447/447 [==============================] - 93s 207ms/step - loss: 5.5113 - accuracy: 0.1007
Epoch 6/90
447/447 [==============================] - 90s 202ms/step - loss: 5.3872 - accuracy: 0.1128
Epoch 7/90
447/447 [==============================] - 94s 211ms/step - loss: 5.2829 - accuracy: 0.1222
Epoch 8/90
447/447 [==============================] - 94s 210ms/step - loss: 5.1875 - accuracy: 0.1284
Epoch 9/90
447/447 [==============================] - 103s 231ms/step - loss: 5.1016 - accuracy: 0.1353
Epoch 10/90
447/447 [==============================] - 102s 229ms/step -

In [4]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
  result = list()
  input = seed_text
  # generate given num words
  for _ in range(n_words):
    # Encode seed text
    encoded = tokenizer.texts_to_sequences([input])[0]
    # Truncate sequences to length of seq
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    # Predict probs for each word
    probs = model.predict_classes(encoded, verbose=0)
    # Map predicted word index to a word
    out_word = ''
    for word, index in tokenizer.word_index.items():
      if index == probs:
        out_word = word
        break

    # Append to input
    input += ' ' + out_word
    result.append(out_word)

  return ' '.join(result)


# open the file and split into lines
file = open_doc('obama_sequences.txt')
lines = file.split('\n')

seq_length = len(lines[0].split()) - 1

# Load model and tokenizer for text generation
model = load_model('model-90.h5')
tokenizer = load(open('tokenizer-90.pkl', 'rb'))

for _ in range(5):
  # Select a seed
  seed_text = lines[randint(0, len(lines))]
  print('Seed: ' + seed_text + '\n')

  generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
  print(generated + '\n')

Seed: america . mr. speaker , mr. vice president , members of congress , distinguished guests , and fellow americans : last month , i went to andrews air force base and welcomed home some of our last troops to serve in iraq . together , we offered a final , proud

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
salute to the polls who believe that we can do it . and we need to teach it with the next 5 years . we should be able to do business . we can do it . but we can do it if we want to work with the world

Seed: reducing teen pregnancy , even reducing violent crime . in states that make it a priority to educate our youngest children , like georgia or oklahoma , studies show students grow up more likely